# ETL-GDP-Data-Pipeline

In this notebook, we will create an ETL pipeline that extracts GDP data from a website, transforms it and loads it into both a CSV file and a database. We will also run a query on the database to find countries with GDPs over 100 billion USD and log the entire process.

## Objectives
- Extract relevant GDP information from a website.
- Transform the data from USD (Millions) to USD (Billions).
- Load the transformed data into a CSV file and a SQLite database.
- Query the database to find countries with GDPs over 100 billion USD.
- Log the entire ETL process.

## 1. Importing Required Libraries

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime

## 2. Defining Functions

### 2.1. Data Extraction Function
This function extracts the required information from the website and saves it to a dataframe. The function returns the dataframe for further processing.

In [13]:
def extract(url, table_attribs):
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    df = pd.DataFrame(columns=table_attribs)
    tables = soup.find_all('tbody')[2]
    rows = tables.find_all('tr')
    for row in rows[3:]:
        cells = row.find_all('td')
        country = cells[0].text
        gdp_usd_millions = cells[1].text
        if gdp_usd_millions != '—':    
            data_dict = {
                "Country": country,
                "GDP_USD_millions": gdp_usd_millions
            }
            df1 = pd.DataFrame(data_dict, index=[0])
            df = pd.concat([df,df1], ignore_index=True)
    return df

### 2.2. Data Transformation Function
This function converts the GDP information from currency format to float value and transforms the information of GDP from USD (Millions) to USD (Billions), rounding to 2 decimal places.

In [16]:
def transform(df):
    GDP_list = df["GDP_USD_millions"].tolist()
    GDP_list = [float(x.replace(',', '')) for x in GDP_list]
    GDP_list = [np.round(x/1000,2) for x in GDP_list]
    df["GDP_USD_millions"] = GDP_list
    df = df.rename(columns={"GDP_USD_millions": "GDP_USD_billions"})
    return df

### 2.3. Loading Data to CSV
This function saves the final dataframe as a CSV file in the provided path.

In [19]:
def load_to_csv(df, csv_path):
    df.to_csv(csv_path, index=False)

### 2.4. Loading Data to Database
This function saves the final dataframe as a database table with the provided name.

In [22]:
def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

### 2.5. Running a Query on the Database
This function runs the stated query on the database table and prints the output on the terminal.

In [27]:
def run_query(query_statement, sql_connection):
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

### 2.6. Logging the Process
This function logs the mentioned message at a given stage of the code execution to a log file.

In [30]:
def log_progress(message):
    timestamp_format = '%Y-%b-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open("./etl_project_log.txt", "a") as f: 
        f.write(timestamp + ' : ' + message + '\n')

## 3. Setting Up the ETL Process
Now that the functions are defined, we will set up the ETL process by calling these functions in sequence.

In [33]:
# Define variables
url = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)'
table_attribs = ["Country", "GDP_USD_millions"]
db_name = 'World_Economies.db'
table_name = 'Countries_by_GDP'
csv_path = './Countries_by_GDP.csv'

# Start the ETL process
log_progress('Preliminaries complete. Initiating ETL process.')

# Extract data
df = extract(url, table_attribs)
log_progress('Data extraction complete. Initiating Transformation process.')

# Transform data
df = transform(df)
log_progress('Data transformation complete. Initiating loading process.')

# Load data to CSV
load_to_csv(df, csv_path)
log_progress('Data saved to CSV file.')

# Load data to Database
sql_connection = sqlite3.connect(db_name)
log_progress('SQL Connection initiated.')

load_to_db(df, sql_connection, table_name)
log_progress('Data loaded to Database as table. Running the query.')

## 4. Querying the Database
We will now run a query to display countries with GDPs over 100 billion USD.

In [37]:
query_statement = f"SELECT * from {table_name} WHERE GDP_USD_billions >= 100"
query_result = run_query(query_statement, sql_connection)
log_progress('Query execution complete. Process complete.')

# Display the query result
query_result

           Country  GDP_USD_billions
0    United States          28781.08
1            China          18532.63
2          Germany           4591.10
3            Japan           4110.45
4            India           3937.01
..             ...               ...
65       Guatemala            110.04
66            Oman            108.93
67        Bulgaria            107.93
68           Kenya            104.00
69       Venezuela            102.33

[70 rows x 2 columns]


## 5. Logging the Process
Finally, let's check the log file to see the recorded steps.

In [40]:
# Read the log file
with open("./etl_project_log.txt", "r") as f:
    log_content = f.read()

print(log_content)

2024-Aug-21-15:17:14 : Preliminaries complete. Initiating ETL process.
2024-Aug-21-15:17:15 : Data extraction complete. Initiating Transformation process.
2024-Aug-21-15:17:15 : Data transformation complete. Initiating loading process.
2024-Aug-21-15:17:15 : Data saved to CSV file.
2024-Aug-21-15:17:15 : SQL Connection initiated.
2024-Aug-21-15:17:15 : Data loaded to Database as table. Running the query.
2024-Aug-21-15:18:37 : Query execution complete. Process complete.

